# Rule-based feature engineering (v2)

This notebook converts AML rule signals into customer-level features.

Inputs:
- `data/interim/master_customers.csv`
- `data/interim/master_transactions.csv`
- `data/raw/labels.csv`

Outputs:
- `data/processed/features_customer_v2.csv` (customer-level matrix)
- quick validation plots vs labels (labeled subset only)

Notes:
- Some rule signals require fields not present in our dataset (e.g., `branch_id`, `counterparty_id`).
  For those, we implement the closest feasible proxy using available columns.


In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path

# Change this if needed: should point to repo root
REPO_ROOT = os.getcwd()  # if notebook launched from repo root
# If notebook is inside /notebooks, uncomment:
# REPO_ROOT = os.path.abspath("..")

REPO_ROOT = Path.cwd().parents[0]  # go from notebooks/ → project root
DATA_DIR = REPO_ROOT / "data"
cust_path   = os.path.join(REPO_ROOT, "data", "interim", "master_customers.csv")
tx_path     = os.path.join(REPO_ROOT, "data", "interim", "master_transactions.csv")
labels_path = os.path.join(REPO_ROOT, "data", "raw", "labels.csv")

cust = pd.read_csv(cust_path)
tx   = pd.read_csv(tx_path)
labels = pd.read_csv(labels_path)

tx["transaction_datetime"] = pd.to_datetime(tx["transaction_datetime"], errors="coerce")

print(cust.shape, tx.shape, labels.shape)
print(tx.columns.tolist())


In [ ]:
# Basic sanity
assert "customer_id" in cust.columns
assert "customer_id" in tx.columns
assert "transaction_datetime" in tx.columns
assert "amount_cad" in tx.columns
assert "channel" in tx.columns

# Helper: get "as-of date" per customer (latest txn time)
cust_last_dt = tx.groupby("customer_id")["transaction_datetime"].max()

def filter_window(tx_df, days: int):
    """
    Keep txns within the last N days per customer based on their last transaction date.
    """
    last_dt = tx_df.groupby("customer_id")["transaction_datetime"].transform("max")
    return tx_df[tx_df["transaction_datetime"] >= (last_dt - pd.to_timedelta(days, unit="D"))].copy()

def safe_ratio(num, den):
    den = den.replace(0, np.nan)
    out = num / den
    return out.fillna(0.0)


In [ ]:
# -----------------------------
# TX-CASH-13 night_cash_activity (30d)  ✅ already seems useful
# -----------------------------
tx_30 = filter_window(tx, 30)

tx_30["is_night"] = tx_30["transaction_datetime"].dt.hour.isin(list(range(22,24)) + list(range(0,6)))
tx_30["is_cash"] = tx_30["cash_indicator"].fillna(0).astype(int) == 1

night_cash = tx_30.groupby("customer_id").apply(
    lambda g: safe_ratio(
        (g["is_night"] & g["is_cash"]).sum(),
        (g["is_cash"]).sum()
    ).iloc[0] if len(g)>0 else 0.0
).rename("night_cash_ratio_30d")

night_cash_cnt = tx_30.groupby("customer_id").apply(
    lambda g: int((g["is_night"] & g["is_cash"]).sum())
).rename("night_cash_txn_count_30d")

cash_cnt_30d = tx_30.groupby("customer_id").apply(
    lambda g: int((g["is_cash"]).sum())
).rename("cash_txn_count_30d")

# -----------------------------
# TX-CASH-11 cash_to_wire_pass_through (proxy)
# Signal intent: cash deposits rapidly exit via wire transfers within ~3d
# We don't have direction (in/out), so proxy:
# "cash volume in last 3d" vs "wire volume in last 3d"
# -----------------------------
tx_3 = filter_window(tx, 3)
cash_amt_3d = tx_3.loc[tx_3["cash_indicator"]==1].groupby("customer_id")["amount_cad"].sum().rename("cash_amt_3d")
wire_amt_3d = tx_3.loc[tx_3["channel"].str.lower().eq("wire")].groupby("customer_id")["amount_cad"].sum().rename("wire_amt_3d")

cash_to_wire_ratio_3d = (wire_amt_3d / (cash_amt_3d.replace(0, np.nan))).fillna(0.0).rename("cash_to_wire_ratio_3d")

# Optional: within 30d context (3d / 30d cash)
cash_amt_30d = tx_30.loc[tx_30["cash_indicator"]==1].groupby("customer_id")["amount_cad"].sum().rename("cash_amt_30d")
cash_to_wire_ratio_3d_30d = (wire_amt_3d / (cash_amt_30d.replace(0, np.nan))).fillna(0.0).rename("cash_to_wire_ratio_3d_30d")

# -----------------------------
# TX-CASH-12 cash_structuring_pattern (proxy)
# True structuring needs branch/location + sub-threshold pattern.
# We lack branch_id, but we DO have city/province and timestamps.
# Proxy feature:
# - count of CASH txns in 30d with amount in "near-threshold band"
# - plus "cash burstiness": max #cash txns in a single day
# We'll pick a threshold band using dataset quantiles (e.g., top 10% of cash amounts)
# -----------------------------
cash_tx = tx_30.loc[tx_30["cash_indicator"]==1].copy()

if len(cash_tx) > 0:
    # choose band: between 80th and 95th percentile of cash amounts
    q80 = cash_tx["amount_cad"].quantile(0.80)
    q95 = cash_tx["amount_cad"].quantile(0.95)
else:
    q80, q95 = 0, 0

cash_tx["near_threshold"] = cash_tx["amount_cad"].between(q80, q95, inclusive="both")

struct_near_count_30d = cash_tx.groupby("customer_id")["near_threshold"].sum().rename("cash_near_threshold_count_30d")

cash_tx["date"] = cash_tx["transaction_datetime"].dt.date
cash_burst_30d = cash_tx.groupby(["customer_id","date"]).size().groupby("customer_id").max().fillna(0).astype(int).rename("cash_max_daily_count_30d")

# -----------------------------
# TX-CASH-14 white_label_atm_usage (proxy)
# True needs known white-label ATM list; we only have merchant_category + channel.
# Proxy:
# - among ABM channel txns, fraction whose merchant_category looks "hotel/casino/convenience"
# This requires merchant_category strings. We'll use keyword matching.
# -----------------------------
tx_30["mc"] = tx_30["merchant_category"].fillna("").astype(str).str.lower()
tx_30["is_abm"] = tx_30["channel"].astype(str).str.lower().eq("abm")

wl_keywords = ["hotel", "motel", "casino", "convenience", "liquor", "gas", "station", "bar"]
tx_30["is_white_label_proxy"] = tx_30["is_abm"] & tx_30["mc"].apply(lambda s: any(k in s for k in wl_keywords))

wl_abm_ratio_30d = tx_30.groupby("customer_id").apply(
    lambda g: safe_ratio(g["is_white_label_proxy"].sum(), g["is_abm"].sum()).iloc[0] if len(g)>0 else 0.0
).rename("white_label_abm_ratio_30d")

wl_abm_count_30d = tx_30.groupby("customer_id")["is_white_label_proxy"].sum().rename("white_label_abm_count_30d")

# -----------------------------
# TX-CASH-15 logistics_worker_cash_volume (customer,30d,relative)
# You DO have occupation_code but not mapping to "logistics".
# Proxy:
# - compute customer cash_amt_30d
# - compare to peers with same occupation_code (z-score)
# -----------------------------
# customer-level cash amt in 30d
cust_cash_amt_30d = cash_tx.groupby("customer_id")["amount_cad"].sum().rename("cash_amt_30d_from_txn")

# merge occupation_code
tmp = cust[["customer_id","occupation_code"]].merge(cust_cash_amt_30d, on="customer_id", how="left")
tmp["cash_amt_30d_from_txn"] = tmp["cash_amt_30d_from_txn"].fillna(0.0)

peer_stats = tmp.groupby("occupation_code")["cash_amt_30d_from_txn"].agg(["mean","std"]).reset_index()
tmp = tmp.merge(peer_stats, on="occupation_code", how="left")
tmp["std"] = tmp["std"].replace(0, np.nan)
tmp["cash_amt_30d_occ_z"] = ((tmp["cash_amt_30d_from_txn"] - tmp["mean"]) / tmp["std"]).fillna(0.0)

cash_amt_30d_occ_z = tmp.set_index("customer_id")["cash_amt_30d_occ_z"].rename("cash_amt_30d_occ_z")

# -----------------------------
# TX-CASH-16 armored_car_cash_volume (proxy)
# Requires channel indicates armored car; not present.
# Proxy:
# - treat very large cash deposits as armored-car-like (top 1% cash amounts)
# - sum of those in 30d
# -----------------------------
if len(cash_tx) > 0:
    q99 = cash_tx["amount_cad"].quantile(0.99)
else:
    q99 = np.inf

cash_tx["very_large_cash"] = cash_tx["amount_cad"] >= q99
armored_cash_sum_30d_proxy = cash_tx.loc[cash_tx["very_large_cash"]].groupby("customer_id")["amount_cad"].sum().rename("armored_cash_sum_30d_proxy")
armored_cash_cnt_30d_proxy = cash_tx.loc[cash_tx["very_large_cash"]].groupby("customer_id").size().rename("armored_cash_cnt_30d_proxy")

# -----------------------------
# TX-WIRE-21 high_risk_jurisdiction_wires (30d, ratio)
# We have transaction country + channel.
# Proxy:
# - for wire/eft, fraction involving a "high-risk" country list (custom list you can tune)
# -----------------------------
high_risk = set(["china","mexico","colombia","hong kong","singapore","south korea","u.a.e","uae"])

tx_30["country_l"] = tx_30["country"].fillna("").astype(str).str.lower()
tx_30["is_wireish"] = tx_30["channel"].astype(str).str.lower().isin(["wire","eft"])
tx_30["is_high_risk_ctry"] = tx_30["is_wireish"] & tx_30["country_l"].isin(high_risk)

high_risk_wire_ratio_30d = tx_30.groupby("customer_id").apply(
    lambda g: safe_ratio(g["is_high_risk_ctry"].sum(), g["is_wireish"].sum()).iloc[0] if len(g)>0 else 0.0
).rename("high_risk_wire_ratio_30d")

high_risk_wire_amt_ratio_30d = tx_30.groupby("customer_id").apply(
    lambda g: safe_ratio(g.loc[g["is_high_risk_ctry"], "amount_cad"].sum(), g.loc[g["is_wireish"], "amount_cad"].sum()).iloc[0] if len(g)>0 else 0.0
).rename("high_risk_wire_amt_ratio_30d")

# -----------------------------
# TX-EMT-31 emt_pass_through (proxy)
# intent: incoming emt depleted by outgoing quickly
# we don't have direction -> proxy:
# - high EMT activity concentration in 3d vs 30d (burst)
# - EMT_3d_amount / EMT_30d_amount
# -----------------------------
emt_3d_amt = tx_3.loc[tx_3["channel"].str.lower().eq("emt")].groupby("customer_id")["amount_cad"].sum().rename("emt_amt_3d")
emt_30d_amt = tx_30.loc[tx_30["channel"].str.lower().eq("emt")].groupby("customer_id")["amount_cad"].sum().rename("emt_amt_30d")

emt_burst_ratio_3d_30d = (emt_3d_amt / (emt_30d_amt.replace(0, np.nan))).fillna(0.0).rename("emt_burst_ratio_3d_30d")

# -----------------------------
# GEO-COR-51 drug_corridor_activity (proxy)
# You have city/province. We'll use a small corridor list, tune later.
# Proxy: count of txns in corridor city list (30d)
# -----------------------------
corridor_cities = set([
    "vancouver","calgary","saskatoon","regina",
    "toronto","london","hamilton"
])
tx_30["city_l"] = tx_30["city"].fillna("").astype(str).str.lower()
tx_30["is_corridor_city"] = tx_30["city_l"].isin(corridor_cities)

corridor_txn_count_30d = tx_30.groupby("customer_id")["is_corridor_city"].sum().rename("corridor_txn_count_30d")
corridor_txn_ratio_30d = tx_30.groupby("customer_id").apply(
    lambda g: safe_ratio(g["is_corridor_city"].sum(), len(g)).iloc[0] if len(g)>0 else 0.0
).rename("corridor_txn_ratio_30d")

# -----------------------------
# BEH-LIFE-113 luxury_spending_vs_income (proxy)
# Use merchant_category keywords: jewelry/designer/luxury
# Feature: luxury_amt_30d / income (from customer)
# -----------------------------
lux_keywords = ["jewelry","designer","luxury","boutique","watch","handbag"]
tx_30["is_lux_proxy"] = tx_30["mc"].apply(lambda s: any(k in s for k in lux_keywords))
lux_amt_30d = tx_30.loc[tx_30["is_lux_proxy"]].groupby("customer_id")["amount_cad"].sum().rename("lux_amt_30d")

cust_income = cust[["customer_id","income"]].copy()
cust_income["income"] = pd.to_numeric(cust_income["income"], errors="coerce").fillna(0.0)

lux_df = cust_income.merge(lux_amt_30d, on="customer_id", how="left")
lux_df["lux_amt_30d"] = lux_df["lux_amt_30d"].fillna(0.0)
lux_spend_income_ratio_30d = (lux_df["lux_amt_30d"] / lux_df["income"].replace(0, np.nan)).fillna(0.0)
lux_spend_income_ratio_30d = pd.Series(lux_spend_income_ratio_30d.values, index=lux_df["customer_id"]).rename("lux_spend_income_ratio_30d")

# Collect features into a single frame
feature_series = [
    night_cash, night_cash_cnt, cash_cnt_30d,
    cash_amt_3d, wire_amt_3d, cash_to_wire_ratio_3d, cash_to_wire_ratio_3d_30d,
    struct_near_count_30d, cash_burst_30d,
    wl_abm_ratio_30d, wl_abm_count_30d,
    cash_amt_30d_occ_z,
    armored_cash_sum_30d_proxy, armored_cash_cnt_30d_proxy,
    high_risk_wire_ratio_30d, high_risk_wire_amt_ratio_30d,
    emt_burst_ratio_3d_30d,
    corridor_txn_count_30d, corridor_txn_ratio_30d,
    lux_spend_income_ratio_30d
]

features_v2 = pd.concat(feature_series, axis=1).reset_index().rename(columns={"index":"customer_id"})
features_v2.head()
